In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [2]:
# 데이터 읽어오기
train_file = "./corpus/ChatbotData.csv"

In [3]:
data = pd.read_csv(train_file, delimiter=',')

In [4]:
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [5]:
features = data['Q'].tolist()
labels = data['label'].tolist()

In [6]:
# 단어 인덱스 시퀀스 벡터g
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index

MAX_SEQ_LEN = 20 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [7]:
# 학습용, 검증용, 테스트용 데이터셋 생성
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(5)
val_ds = ds.skip(train_size).take(val_size).batch(5)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(5)

In [8]:
# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 16
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1 # 전체 단어 수

In [9]:
# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=16,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=16,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=16,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

In [10]:
# 3, 4, 5-gram 이후 겹치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(64, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)
predictions = Dense(3, activation=tf.nn.softmax)(logits)

In [11]:
# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [12]:
# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
1656/1656 [==============================] - 17s 8ms/step - loss: 0.9397 - accuracy: 0.5436 - val_loss: 0.7970 - val_accuracy: 0.6197
Epoch 2/5
1656/1656 [==============================] - 11s 7ms/step - loss: 0.7149 - accuracy: 0.6918 - val_loss: 0.4698 - val_accuracy: 0.8397
Epoch 3/5
1656/1656 [==============================] - 12s 7ms/step - loss: 0.5153 - accuracy: 0.8090 - val_loss: 0.2969 - val_accuracy: 0.9137
Epoch 4/5
1656/1656 [==============================] - 13s 8ms/step - loss: 0.3790 - accuracy: 0.8682 - val_loss: 0.2049 - val_accuracy: 0.9289
Epoch 5/5
1656/1656 [==============================] - 13s 8ms/step - loss: 0.3051 - accuracy: 0.9003 - val_loss: 0.1540 - val_accuracy: 0.9522


In [13]:
# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

237/237 [==============================] - 1s 3ms/step - loss: 0.1870 - accuracy: 0.9501
Accuracy: 95.008463
loss: 0.187000


In [14]:
tf.__version__

'2.5.0'